<a href="https://colab.research.google.com/github/diegohugo570/backup-python/blob/main/01_Aula_Pr%C3%A1tica_LangGraph_do_Zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-openai langgraph
!pip install --upgrade google-search-results

In [ ]:
# Bibliotecas
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Annotated
from typing import List

from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from bs4 import BeautifulSoup
import requests

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Funções Auxiliares

In [ ]:
def get_google_results_links(query: str, num_results: int = 1):
  url = "https://serpapi.com/search"
  params = {
      "engine": "google",
      "q": query,
      "api_key": userdata.get("SERP_API_KEY")
  }

  response = requests.get(url, params=params)
  return list(map(lambda x: x.get("link", ""), response.json()["related_questions"]))[:num_results]

In [ ]:
def extract_text_from_site(url):
  soup = BeautifulSoup(requests.get(url).text)
  texto = ""
  for tag in soup.find_all(["p", "h1", "h2", "h3", "h4", "h5", "h6"]):
    texto += tag.get_text() + "\n"
  return texto

In [ ]:
def format_history(history, n: int = -1):
    if not history:
        return ""

    response = ""
    total_messages = len(history)
    total_pairs = total_messages // 2

    if n == -1 or n > total_pairs:
        n = total_pairs
    elif n <= 0:
        n = 0

    start_idx = total_messages - (n * 2)
    if start_idx < 0:
        start_idx = 0

    for k in range(start_idx, total_messages, 2):
        human = history[k]
        response += f"- Usuário: {human.content}\n"
        if k + 1 < total_messages:
            ai = history[k + 1]
            response += f"- Você: {ai.content}\n"
    return response

## Definição do Estado

In [ ]:
class State(TypedDict):
  mensagem: Annotated[str, "Mensagem do usuário"]
  score: Annotated[int, "Nota do avaliador"]
  query: Annotated[str, "Query reescrita pelo agente"]
  resultado_pesquisa: Annotated[str, "Resultados da pesquisa"]
  fitness: Annotated[bool, "A pergunta é sobre fitness ou não?"]
  historico: Annotated[List[AnyMessage], "Histórico de perguntas"]
  resposta: Annotated[str, "Resposta do agente"]

# Definição das Saídas Estruturadas

In [ ]:
class RouterResponse(BaseModel):
  cot: List[str] = Field(description="Cadeia de pensamento do agente")
  fitness: bool = Field(description="Pergunta relacionada a fitness ou não")

In [ ]:
class QueryRewriterResponse(BaseModel):
  cot: List[str] = Field(description="Cadeia de pensamento do agente")
  query: str = Field(description="Query reescrita otimizada")

In [ ]:
class VerificadorResponse(BaseModel):
  cot: List[str] = Field(description="Cadeia de pensamento do agente do agente")
  score: int = Field(description="Número entre 0 e 10")

## Criar os nós

In [ ]:
def router_node(state: State) -> State:
  mensagem = state["mensagem"]
  historico = state["historico"]

  prompt = """
  <Especialista>
  Você é um agente especialista em classificação de intenção de mensagens. Sua tarefa é analisar conversas entre usuários e o assistente e identificar se a intenção do usuário é tirar dúvidas sobre conteúdos **fitness** (saúde, bem-estar, nutrição, emagrecimento, treino, academia, corpo etc.) ou **não**.

  Seu papel é exclusivamente classificar, sem responder perguntas ou dar opiniões fora da análise.
  </Especialista>

  <Contexto>
  Você receberá duas entradas:
  1. Histórico da conversa em formato de lista de mensagens anteriores (pode estar vazio).
  2. Última mensagem enviada pelo usuário (a que deve ser avaliada com mais peso).

  Com base no conteúdo semântico e lexical dessas mensagens, você deverá:
  - Avaliar se o usuário tem a intenção de tirar dúvidas sobre conteúdos fitness.
  - Gerar uma **cadeia de pensamento lógica** (em linguagem natural) justificando sua decisão.
  - Retornar um JSON contendo a cadeia de pensamento e um booleano indicando o resultado da classificação.
  </Contexto>

  <Instrução>
  Leia o histórico e a última mensagem. Analise o conteúdo considerando palavras-chave, contexto semântico e intenção implícita. Considere sinônimos e termos relacionados ao universo fitness.

  Se houver qualquer sinal que indique que a intenção é relacionada a fitness, classifique como `true`. Caso contrário, `false`.

  Retorne apenas o JSON com os seguintes campos:
  - `cadeia_de_pensamento`: explicação lógica do raciocínio seguido.
  - `fitness`: booleano indicando se é sobre fitness ou não.
  </Instrução>

  <Tom de Comunicação>
  Técnico, direto e objetivo. Nada além do necessário.
  </Tom de Comunicação>

  <Histórico de mensagens>
  {historico}
  </Histórico de mensagens>

  <Última mensagem enviada pelo usuário>
  {mensagem}
  </Última mensagem enviada pelo usuário>
  """

  prompt_template = PromptTemplate.from_template(prompt)

  llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.2)
  llm_strucutured = llm.with_structured_output(RouterResponse)

  chain = prompt_template | llm_strucutured

  resposta = chain.invoke({"historico": format_history(historico), "mensagem": mensagem})

  return {"fitness": resposta.fitness}

In [ ]:
def verificador_coerencia_node(state: State) -> State:
  mensagem = state["mensagem"]
  historico = state["historico"]
  resultado_pesquisa = state["resultado_pesquisa"]

  prompt = """
 <Verificador>
  Você é um agente especialista em verificação de coerência entre a dúvida do usuário e os resultados encontrados em uma pesquisa do Google.

  Sua função é analisar se os resultados retornados pela pesquisa são suficientes e coerentes para responder à dúvida ou necessidade expressa pelo usuário. Você não deve responder à dúvida, apenas avaliar se é possível respondê-la com o material disponível.
  </Verificador>

  <Contexto>
  Você receberá três entradas:
  1. `historico`: uma lista contendo as mensagens anteriores trocadas entre o usuário e o assistente.
  2. `mensagem_atual`: a última mensagem enviada pelo usuário, que deve ser analisada com mais peso.
  3. `resultado_pesquisa`: uma lista de trechos retornados da busca no Google, que podem conter partes de artigos, títulos, descrições e links.

  Seu trabalho é verificar se esses resultados são suficientes para responder, com qualidade, a solicitação feita pelo usuário. Não considere apenas palavras-chave, mas a **qualidade, relevância, atualidade e coerência contextual** dos conteúdos apresentados.
  </Contexto>

  <Instrução>
  Analise o conteúdo da última mensagem do usuário, complementando com o histórico se necessário. Em seguida, avalie os `resultado_pesquisa` para verificar se:
  - Existe **relação direta** entre os conteúdos encontrados e o que foi perguntado.
  - Os resultados são **suficientemente informativos** para permitir uma resposta clara, útil e bem embasada.

  Atribua um `score` de 0 a 10:
  - 0 = nenhum resultado é útil
  - 10 = todos os elementos da dúvida podem ser plenamente respondidos com os resultados

  Retorne um JSON com os campos:
  - `cot`: cadeia de pensamento que justifica a nota atribuída
  - `score`: valor inteiro entre 0 e 10
  </Instrução>

  <Tom de Comunicação>
  Preciso, técnico e lógico.
  </Tom de Comunicação>

  <Histórico de mensagens>
  {historico}
  </Histórico de mensagens>

  <Última mensagem enviada pelo usuário>
  {mensagem}
  </Última mensagem enviada pelo usuário>

  <Resultados Pesquisa>
  {resultado_pesquisa}
  </Resultados Pesquisa>
  """

  prompt_template = PromptTemplate.from_template(prompt)

  llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.2)
  llm_strucutured = llm.with_structured_output(VerificadorResponse)

  chain = prompt_template | llm_strucutured

  resposta = chain.invoke({"historico": format_history(historico), "mensagem": mensagem, "resultado_pesquisa": resultado_pesquisa})

  return {"score": resposta.score}

In [ ]:
def query_rewriter_node(state: State) -> State:
  mensagem = state["mensagem"]
  historico = state["historico"]
  resultado_pesquisa = state["resultado_pesquisa"]
  query = state["query"]


  invoke_chain_dict = {"historico": format_history(historico), "mensagem": mensagem, "resultado_pesquisa": resultado_pesquisa}

  prompt = """
  <Especialista>
  Você é um agente especialista em reescrita de queries para buscas no Google. Sua função é otimizar a query do usuário para que os resultados da pesquisa sejam mais relevantes, precisos e úteis.
  </Especialista>

  <Contexto>
  Você receberá três entradas:
  1. `historico`: uma lista com as mensagens anteriores trocadas entre o usuário e o assistente.
  2. `mensagem_atual`: a última mensagem enviada pelo usuário, com maior peso na análise.
  3. `resultado_pesquisa`: lista com os resultados da última busca, considerados ruins ou pouco relevantes.

  Sua tarefa é identificar por que os resultados anteriores foram insatisfatórios e reescrever a query com mais clareza, especificidade e intenção explícita, considerando o que o usuário realmente quer saber.
  </Contexto>

  <Instrução>
  Analise a intenção do usuário com base no histórico e na última mensagem. Em seguida, avalie os resultados da busca para entender por que não atenderam à necessidade (ex: genéricos, vagos, comerciais, irrelevantes).

  Depois disso, reescreva a query com melhorias que podem incluir:
  - Uso de palavras-chave mais específicas
  - Inclusão de contexto (ex: "em 2024", "para iniciantes", "com evidência científica")
  - Filtros úteis (ex: `site:.gov`, `site:.org`, `"filetype:pdf"`, etc.)
  - Reformulação de expressões ambíguas

  Sempre retorne um JSON com:
  - `cot`: explicação lógica (em linguagem natural) da cadeia de raciocínio usada para reescrever a query.
  - `query`: a nova query otimizada para busca no Google.

  Exclua qualquer outro conteúdo que não esteja nesse formato.
  </Instrução>

  <Tom de Comunicação>
  Técnico, direto e lógico.
  </Tom de Comunicação>

  <Histórico de mensagens>
  {historico}
  </Histórico de mensagens>

  <Última mensagem enviada pelo usuário>
  {mensagem}
  </Última mensagem enviada pelo usuário>

  <Resultados Pesquisa>
  Você já tentou buscar os documentos uma vez e o resultado foi ruim. Ele está abaixo:

  # Resultados
  {resultado_pesquisa}
  </Resultados Pesquisa>
  """

  if query:
    prompt += """

    <Sua query anterior>
    Você já tentou otimizar a busca uma vez e teve um resultado ruim.

    # Query anterior
    {query}
    </Sua query anterior>
    """
    invoke_chain_dict["query"] = query

  prompt_template = PromptTemplate.from_template(prompt)

  llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.2)
  llm_strucutured = llm.with_structured_output(QueryRewriterResponse)

  chain = prompt_template | llm_strucutured

  resposta = chain.invoke(invoke_chain_dict)

  return {"query": resposta.query}

In [ ]:
def google_agent_node(state: State) -> State:
  mensagem = state["mensagem"]
  query = state["query"]

  pergunta = query if query else mensagem

  urls = get_google_results_links(pergunta)
  text = ""
  for url in urls:
    text += extract_text_from_site(url)
  return {"resultado_pesquisa": text}

In [ ]:
def specialist_node(state: State) -> State:
  mensagem = state["mensagem"]
  historico = state["historico"]
  resultado_pesquisa = state["resultado_pesquisa"]

  invoke_chain_dict = {"historico": format_history(historico), "mensagem": mensagem}

  prompt = """
  <Especialista>
  Você é um especialista em **fitness**, nutrição, treinos, saúde corporal, emagrecimento, hipertrofia e bem-estar físico. Seu papel é responder **exclusivamente** dentro desse universo.

  Você analisa sempre:
  1. O **histórico da conversa** com o usuário;
  2. A **última mensagem** enviada por ele.

  Você nunca sai do tema fitness. Quando o usuário pergunta algo fora do escopo, você responde com **bom humor** e **traz o assunto de volta para o fitness** usando analogias criativas.
  </Especialista>

  <Contexto>
  Entrada:
  - `historico`: lista com as mensagens anteriores trocadas com o usuário;
  - `mensagem_atual`: a última mensagem enviada.

  A saída deve ser **apenas uma resposta em linguagem natural para o usuário**, com base nas duas entradas.

  Se a mensagem estiver relacionada ao universo fitness:
  - Responda com clareza, entusiasmo e objetividade.

  Se estiver **fora do tema fitness**:
  - Brinque com o assunto;
  - Crie uma ponte divertida e **volte para o universo fitness**;
  - Nunca diga que não pode responder. Apenas transforme o tema.

  **Não** retorne JSON, tags ou estrutura auxiliar. Somente a resposta final do especialista, pronta para ser enviada ao usuário.
  </Contexto>

  <Instrução>
  1. Analise a intenção do usuário com base no histórico e na última mensagem.
  2. Gere **apenas a resposta textual** do especialista, no tom de um coach motivador, divertido e obcecado por saúde e performance física.
  </Instrução>

  <Tom de Comunicação>
  Animado, direto, divertido e 100% fitness.
  </Tom de Comunicação>

  <Histórico de mensagens>
  {historico}
  </Histórico de mensagens>

  <Última mensagem enviada pelo usuário>
  {mensagem}
  </Última mensagem enviada pelo usuário>
  """

  if resultado_pesquisa:
    prompt += """

    <Fonte de conhecimento>
    Você deve responder a mensagem do usuário com base na fonte de conhecimento.

    # Fonte de conhecimento
    {resultado_pesquisa}
    </Fonte de conhecimento>
    """
    invoke_chain_dict["resultado_pesquisa"] = resultado_pesquisa

  prompt_template = PromptTemplate.from_template(prompt)

  llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.2)

  chain = prompt_template | llm

  resposta = chain.invoke(invoke_chain_dict)

  state["historico"].append(HumanMessage(content = mensagem))
  state["historico"].append(AIMessage(content = resposta.content))

  return {"resposta": resposta.content, "historico": historico}

## Definir os Nós Condicionais

In [ ]:
def router_decider_node(state: State) -> State:
  if state["fitness"]:
    return "google_agent_node"

  return "specialist_node"

In [ ]:
def verificador_decider_node(state: State) -> State:
  if state["score"] >= 7:
    return "specialist_node"

  return "query_rewriter_node"

## Construindo o grafo

In [ ]:
builder = StateGraph(State)

builder.add_node("router_node", router_node)
builder.add_node("verificador_coerencia_node", verificador_coerencia_node)
builder.add_node("query_rewriter_node", query_rewriter_node)
builder.add_node("google_agent_node", google_agent_node)
builder.add_node("specialist_node", specialist_node)

builder.add_edge(START, "router_node")
builder.add_conditional_edges("router_node", router_decider_node)
builder.add_edge("google_agent_node", "verificador_coerencia_node")
builder.add_edge("query_rewriter_node", "google_agent_node")
builder.add_conditional_edges("verificador_coerencia_node", verificador_decider_node)
builder.add_edge("specialist_node", END)

checkpointer = MemorySaver()
app = builder.compile(checkpointer = checkpointer)

In [ ]:
class State(TypedDict):
  mensagem: Annotated[str, "Mensagem do usuário"]
  score: Annotated[int, "Nota do avaliador"]
  query: Annotated[str, "Query reescrita pelo agente"]
  resultado_pesquisa: Annotated[str, "Resultados da pesquisa"]
  fitness: Annotated[bool, "A pergunta é sobre fitness ou não?"]
  historico: Annotated[List[AnyMessage], "Histórico de perguntas"]
  resposta: Annotated[str, "Resposta do agente"]

In [ ]:
config = {"configurable": {"thread_id": "5"}}

initial_state = {
    "mensagem": "Opa",
    "score": 0,
    "query": "",
    "resultado_pesquisa": "",
    "fitness": True,
    "historico": [],
    "resposta": ""
}

In [ ]:
for output in app.stream(initial_state, config=config):
    for key, value in output.items():
        print(f"{key}: {value}")

resposta = value['resposta']

print(resposta)

router_node: {'fitness': False}
specialist_node: {'resposta': 'E aí, campeão! Pronto para levantar o ânimo e dar um gás nos treinos? Vamos falar sobre como turbinar sua rotina de exercícios ou ajustar a alimentação para alcançar seus objetivos! O que você tem em mente? 💪🥦', 'historico': [HumanMessage(content='Opa', additional_kwargs={}, response_metadata={}), AIMessage(content='E aí, campeão! Pronto para levantar o ânimo e dar um gás nos treinos? Vamos falar sobre como turbinar sua rotina de exercícios ou ajustar a alimentação para alcançar seus objetivos! O que você tem em mente? 💪🥦', additional_kwargs={}, response_metadata={})]}
E aí, campeão! Pronto para levantar o ânimo e dar um gás nos treinos? Vamos falar sobre como turbinar sua rotina de exercícios ou ajustar a alimentação para alcançar seus objetivos! O que você tem em mente? 💪🥦


In [ ]:
for output in app.stream({"mensagem": "qual o melhor método de emagrecimento? quero ficar secoooo"}, config=config):
    for key, value in output.items():
        print(f"{key}: {value}")

resposta = value['resposta']

print(resposta)

router_node: {'fitness': True}
google_agent_node: {'resultado_pesquisa': 'Dieta para secar e perder barriga (com cardápio)\nPara perder a barriga, é importante utilizar algumas mudanças nos hábitos alimentares, devendo diminuir o consumo principalmente de alimentos ricos em açúcares e gorduras, e iniciar ou intensificar a prática de atividade física, como musculação, corrida, caminhada ou bicicleta, por exemplo.\nÉ fundamental que a alimentação seja rica em fibras, proteínas e alimentos probióticos, pois favorecem o bom funcionamento do intestino, melhoram a sensação de saciedade e favorecem o ganho de massa muscular, o que é fundamental para acelerar a eliminação de gordura acumulada no organismo.\nA consulta com o nutricionista é importante para que seja feita uma avaliação da composição corporal, sendo verificado os níveis de gordura e massa muscular, e possa ser indicada a melhor alimentação. É importante que a dieta adequada seja acompanhada pela prática de atividade física que de